# How to use Amazon Forecast

Helps advanced users start with Amazon Forecast quickly. The demo notebook runs through a typical end to end usecase for a simple timeseries forecasting scenario. 

Prerequisites: 
[AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/installing.html) . 

For more informations about APIs, please check the [documentation](https://docs.aws.amazon.com/forecast/latest/dg/what-is-forecast.html)

## Table Of Contents
* [Setting up](#setup)
* [Test Setup - Running first API](#hello)
* [Forecasting Example with Amazon Forecast](#forecastingExample)

**Read Every Cell FULLY before executing it**


## Setup <a class="anchor" id="setup"></a>

In [ ]:
import sys
import os
import time

import boto3

# importing forecast notebook utility from notebooks/common directory
sys.path.insert( 0, os.path.abspath("../../common") )
import util

Configure the S3 bucket name and region name for this lesson.

- If you don't have an S3 bucket, create it first on S3.
- Although we have set the region to us-west-2 as a default value below, you can choose any of the regions that the service is available in.

In [ ]:
text_widget_bucket = util.create_text_widget( "bucketName", "input your S3 bucket name" )
text_widget_region = util.create_text_widget( "region", "input region name.", default_value="us-west-2" )

In [ ]:
bucketName = text_widget_bucket.value
assert bucketName, "bucket_name not set."

region = text_widget_region.value
assert region, "region not set."

In [ ]:
session = boto3.Session(region_name=region) 

forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')

## Forecasting with Amazon Forecast<a class="anchor" id="forecastingExample"></a>
### Preparing your Data

In Amazon Forecast , a dataset is a collection of file(s) which contain data that is relevant for a forecasting task. A dataset must conform to a schema provided by Amazon Forecast. 

For this exercise, we use the individual household electric power consumption dataset. (Dua, D. and Karra Taniskidou, E. (2017). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.) We aggregate the usage data hourly. 

# Data Type

Amazon forecast can import data from Amazon S3. We first explore the data locally to see the fields

In [ ]:
import pandas as pd
df = pd.read_csv("../../common/data/item-demand-time.csv", dtype = object)
df.head(3)

Now upload the data to S3. But before doing that, go into your AWS Console, select S3 for the service and create a new bucket inside the `Oregon` or `us-west-2` region. Use that bucket name convention of `amazon-forecast-unique-value-data`. The name must be unique, if you get an error, just adjust until your name works, then update the `bucketName` cell below.

In [ ]:
s3 = session.client('s3')

In [ ]:
key="elec_data/item-demand-time.csv"

In [ ]:
s3.upload_file(Filename="../../common/data/item-demand-time.csv", Bucket=bucketName, Key=key)

In [ ]:
# Create the role to provide to Amazon Forecast.
role_name = "ForecastNotebookRole-AutoML"
role_arn = util.get_or_create_iam_role( role_name = role_name )

### CreateDataset

More details about `Domain` and dataset type can be found on the [documentation](https://docs.aws.amazon.com/forecast/latest/dg/howitworks-domains-ds-types.html) . For this example, we are using [CUSTOM](https://docs.aws.amazon.com/forecast/latest/dg/custom-domain.html) domain with 3 required attributes `timestamp`, `target_value` and `item_id`. Also for your project name, update it to reflect your name in a lowercase format.

In [ ]:
DATASET_FREQUENCY = "H" 
TIMESTAMP_FORMAT = "yyyy-MM-dd hh:mm:ss"

In [ ]:
project = 'workshop_forecastdemo_1' # Replace this with a unique name here, make sure the entire name is < 30 characters.
datasetName= project+'_ds'
datasetGroupName= project +'_gp'
s3DataPath = "s3://"+bucketName+"/"+key

In [ ]:
datasetName

### Schema Definition 
### We are defining the attributes for the model 

In [ ]:
# Specify the schema of your dataset here. Make sure the order of columns matches the raw data files.
schema ={
   "Attributes":[
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"target_value",
         "AttributeType":"float"
      },
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      }
   ]
}

response=forecast.create_dataset(
                    Domain="CUSTOM",
                    DatasetType='TARGET_TIME_SERIES',
                    DatasetName=datasetName,
                    DataFrequency=DATASET_FREQUENCY, 
                    Schema = schema
                   )
datasetArn = response['DatasetArn']

In [ ]:
create_dataset_group_response = forecast.create_dataset_group(DatasetGroupName=datasetGroupName,
                                                              Domain="CUSTOM",
                                                              DatasetArns= [datasetArn]
                                                             )
datasetGroupArn = create_dataset_group_response['DatasetGroupArn']

If you have an existing datasetgroup, you can update it using **update_dataset_group** to update dataset group.

In [ ]:
forecast.describe_dataset_group(DatasetGroupArn=datasetGroupArn)

### Create Data Import Job
Brings the data into Amazon Forecast system ready to forecast from raw data. 

In [ ]:
datasetImportJobName = 'EP_AML_DSIMPORT_JOB_TARGET'
ds_import_job_response=forecast.create_dataset_import_job(DatasetImportJobName=datasetImportJobName,
                                                          DatasetArn=datasetArn,
                                                          DataSource= {
                                                              "S3Config" : {
                                                                 "Path":s3DataPath,
                                                                 "RoleArn": role_arn
                                                              } 
                                                          },
                                                          TimestampFormat=TIMESTAMP_FORMAT
                                                         )

In [ ]:
ds_import_job_arn=ds_import_job_response['DatasetImportJobArn']
print(ds_import_job_arn)

Check the status of dataset, when the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on the data size. It can take 10 mins to be **ACTIVE**. This process will take 5 to 10 minutes.

In [ ]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_dataset_import_job(DatasetImportJobArn=ds_import_job_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

In [ ]:
forecast.describe_dataset_import_job(DatasetImportJobArn=ds_import_job_arn)

### Create Predictor with customer forecast horizon

Forecast horizon is the number of number of time points to predicted in the future. For weekly data, a value of 12 means 12 weeks. Our example is hourly data, we try forecast the next day, so we can set to 24.

If we are not sure which recipe will perform best, we can utilise the Auto ML option that the SDK offers.

In [ ]:
predictorName = project+'_autoML'

In [ ]:
forecastHorizon = 24

In [ ]:
algorithmArn = 'arn:aws:forecast:::algorithm/ETS'

In [ ]:
create_predictor_response=forecast.create_predictor(PredictorName=predictorName, 
                                                  ForecastHorizon=forecastHorizon,
                                                  PerformAutoML=True,
                                                  PerformHPO=False,
                                                  EvaluationParameters= {"NumberOfBacktestWindows": 1, 
                                                                         "BackTestWindowOffset": 24}, 
                                                  InputDataConfig= {"DatasetGroupArn": datasetGroupArn},
                                                  FeaturizationConfig= {"ForecastFrequency": "H", 
                                                                        "Featurizations": 
                                                                        [
                                                                          {"AttributeName": "target_value", 
                                                                           "FeaturizationPipeline": 
                                                                            [
                                                                              {"FeaturizationMethodName": "filling", 
                                                                               "FeaturizationMethodParameters": 
                                                                                {"frontfill": "none", 
                                                                                 "middlefill": "zero", 
                                                                                 "backfill": "zero"}
                                                                              }
                                                                            ]
                                                                          }
                                                                        ]
                                                                       }
                                                 )

In [ ]:
predictorArn=create_predictor_response['PredictorArn']

Check the status of the predictor. When the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 10 mins to more than one hour to be **ACTIVE**.

In [ ]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_predictor(PredictorArn=predictorArn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

### Get Error Metrics

Let's get the accuracy metrics of the predicto we just created using Auto ML. The response will be a dictionary with all available recipes. Auto ML works out the best one for our predictor.

In [ ]:
forecast.get_accuracy_metrics(PredictorArn=predictorArn)

### Create Forecast

Now create a forecast using the model that was trained.

In [ ]:
forecastName= project+'_aml_forecast'

In [ ]:
create_forecast_response=forecast.create_forecast(ForecastName=forecastName,
                                                  PredictorArn=predictorArn)
forecastArn = create_forecast_response['ForecastArn']

Check the status of the forecast process, when the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 10 mins to more than one hour to be **ACTIVE**. There's no output here, but that is fine as long as the * is there.

In [ ]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_forecast(ForecastArn=forecastArn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

### Get Forecast

Once created, the forecast results are ready and you view them. 

In [ ]:
forecastResponse = forecastquery.query_forecast(
    ForecastArn=forecastArn,
   Filters={"item_id":"client_12"}
)
print(forecastResponse)

# Export Forecast

You can export forecast to s3 bucket. To do so an role with s3 put access is needed, but this has already been created.

In [ ]:
forecastExportName= project+'_aml_forecast_export'

In [ ]:
outputPath="s3://"+bucketName+"/output"

In [ ]:
forecast_export_response = forecast.create_forecast_export_job(
                                                                ForecastExportJobName = forecastExportName,
                                                                ForecastArn=forecastArn, 
                                                                Destination = {
                                                                   "S3Config" : {
                                                                       "Path":outputPath,
                                                                       "RoleArn": role_arn
                                                                   } 
                                                                }
                                                              )

In [ ]:
forecastExportJobArn = forecast_export_response['ForecastExportJobArn']

In [ ]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_forecast_export_job(ForecastExportJobArn=forecastExportJobArn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

Check s3 bucket for results

In [ ]:
s3.list_objects(Bucket=bucketName,Prefix="output")

# Cleanup

Once we have completed the above steps, we can start to cleanup the resources we created. All delete jobs, except for `delete_dataset_group` are asynchronous, so we have added the helpful `wait_till_delete` function. 
Resource Limits documented <a href="https://docs.aws.amazon.com/forecast/latest/dg/limits.html">here</a>.

In [ ]:
# Delete forecast export for both algorithms
util.wait_till_delete(lambda: forecast.delete_forecast_export_job(ForecastExportJobArn = forecastExportJobArn))

In [ ]:
# Delete forecast
util.wait_till_delete(lambda: forecast.delete_forecast(ForecastArn = forecastArn))

In [ ]:
# Delete predictor
util.wait_till_delete(lambda: forecast.delete_predictor(PredictorArn = predictorArn))

In [ ]:
# Delete Import
util.wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn=ds_import_job_arn))

In [ ]:
# Delete the dataset
util.wait_till_delete(lambda: forecast.delete_dataset(DatasetArn=datasetArn))

In [ ]:
# Delete Dataset Group
util.wait_till_delete(lambda: forecast.delete_dataset_group(DatasetGroupArn=datasetGroupArn))

In [ ]:
# Delete IAM role
util.delete_iam_role( role_name )